<a href="https://colab.research.google.com/github/MohamadHusseinIsmail/Anghami-Deliverables/blob/main/Subscription_analytics_mini_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# I am doing this mini project to convey my skills in Data cleaning, preparation, and transformation using using python.


import pandas as pd
users = pd.read_csv("/content/users.csv", parse_dates=["signup_date"])
subscriptions = pd.read_csv("/content/subscriptions.csv", parse_dates=["start_date", "end_date"])
payments = pd.read_csv("/content/payments.csv", parse_dates=["payment_date"])
sessions = pd.read_csv("/content/streaming_sessions.csv", parse_dates=["session_date"])
content = pd.read_csv("/content/content.csv")
plans = pd.read_csv("/content/plans.csv")



In [ ]:
df.info()
df.head()
df.isna().sum()
# Here the Datasets are already clean but I am double checking just in case :)

In [ ]:
payments["amount_usd"] = pd.to_numeric(payments["amount_usd"], errors="coerce")
sessions["minutes_listened"] = pd.to_numeric(
    sessions["minutes_listened"], errors="coerce"
)
# Here I am fixing the types of amount_usd and minutesListened  variables to make them numeric

In [ ]:
users = users.drop_duplicates(subset=["user_id"])
plans = plans.drop_duplicates(subset=["plan_id"])
subscriptions = subscriptions.drop_duplicates(subset=["subscription_id"])
payments = payments.drop_duplicates(subset=["payment_id"])
sessions = sessions.drop_duplicates(subset=["session_id"])
#Here I am checking for any possible duplicate

In [ ]:
users = users.dropna(subset=["user_id", "signup_date"])
payments = payments.dropna(subset=["payment_date", "amount_usd"])
subscriptions = subscriptions.dropna(subset=["user_id", "plan_id", "start_date"])
payments = payments.dropna(subset=["payment_date", "amount_usd"])
sessions = sessions.dropna(subset=["user_id", "content_id", "session_date"])
sessions["minutes_listened"] = sessions["minutes_listened"].fillna(0)
# Here I want to handle any missing values in all the six datasets

In [ ]:
sessions = sessions[sessions["minutes_listened"] >= 0]
# emphasizing on not having negative listening time

In [ ]:
payments = payments[
    (payments["amount_usd"] > 0) | (payments["payment_type"] == "refund")
]
# No invalid payments here

In [ ]:
subscriptions = subscriptions[
    subscriptions["user_id"].isin(users["user_id"])
]
payments = payments[
    payments["subscription_id"].isin(subscriptions["subscription_id"])
]
sessions = sessions[
    sessions["user_id"].isin(users["user_id"]) &
    sessions["content_id"].isin(content["content_id"])
]

In [ ]:
users["device_os"] = users["device_os"].str.lower().str.strip()
users["acquisition_channel"] = users["acquisition_channel"].str.lower().str.strip()
#standardizing categorical columns

In [ ]:
assert users["user_id"].isna().sum() == 0
assert payments["amount_usd"].sum() >= 0
assert sessions["minutes_listened"].sum() >= 0

# final logical test


In [ ]:
payments["month"] = payments["payment_date"].dt.to_period("M")

monthly_revenue = (
    payments
    .groupby(["user_id", "month"], as_index=False)
    .agg(total_revenue=("amount_usd", "sum"))
)


print(monthly_revenue.head(10))


In [ ]:
print(
    monthly_revenue
    .sort_values(["user_id", "month"])
    .to_string(index=False)
)


In [ ]:
# Monthly Recurring Revenue (MRR)
#The refunds are already negative or filtered
mrr = (
    monthly_revenue
    .groupby("month", as_index=False)
    .agg(MRR=("total_revenue", "sum"))
)

print(mrr.to_string(index=False))


In [ ]:
subscriptions["churn_month"] = subscriptions["end_date"].dt.to_period("M")
subscriptions["start_month"] = subscriptions["start_date"].dt.to_period("M")
active_subs = (
    subscriptions
    .groupby("start_month", as_index=False)
    .agg(active_subscriptions=("subscription_id", "count"))
    .rename(columns={"start_month": "month"})
)
print(active_subs)
churned_subs = (
    subscriptions[subscriptions["is_churned"] == 1]
    .groupby("churn_month", as_index=False)
    .agg(churned_subscriptions=("subscription_id", "count"))
    .rename(columns={"churn_month": "month"})
)
print(churned_subs)

In [ ]:
churn_rate = (
    active_subs
    .merge(churned_subs, on="month", how="left")
    .fillna(3)
)

churn_rate["churn_rate"] = (
    churn_rate["churned_subscriptions"]
    / churn_rate["active_subscriptions"]
)

print(churn_rate.to_string(index=False))


In [ ]:
ltv = (
    monthly_revenue
    .groupby("user_id", as_index=False)
    .agg(LTV=("total_revenue", "sum"))
)

print(ltv.head(10).to_string(index=False))
#Toook the first 10 as a sample output

In [ ]:
assert monthly_revenue["total_revenue"].sum() == payments["amount_usd"].sum()
# Active paying users per month
active_users = (
    monthly_revenue
    .query("total_revenue > 0")
    .groupby("month", as_index=False)
    .agg(active_users=("user_id", "nunique"))
)

# ARPU calculated here
arpu = (
    mrr
    .merge(active_users, on="month")
)

arpu["ARPU"] = arpu["MRR"] / arpu["active_users"]

print(arpu[["month", "MRR", "active_users", "ARPU"]].to_string(index=False))


In [ ]:
users["cohort"] = users["signup_date"].dt.to_period("M")
sessions["month"] = sessions["session_date"].dt.to_period("M")

cohort_data = sessions.merge(
    users[["user_id", "cohort"]],
    on="user_id"
)
cohort_size = (
    cohort_data
    .groupby("cohort")["user_id"]
    .nunique()
)
cohort_retention = (
    cohort_data
    .groupby(["cohort", "month"])["user_id"]
    .nunique()
    .reset_index()
)

cohort_retention["cohort_size"] = cohort_retention["cohort"].map(cohort_size)
cohort_retention["retention_rate"] = (
    cohort_retention["user_id"] / cohort_retention["cohort_size"]
)

print(cohort_retention.head(10).to_string(index=False))

